In [5]:
import os

In [24]:
## ARCPY CSV_TO_FC FUNC
def join_csv_to_fc(in_fc, join_table, in_field, join_field, join_field_len, out_gdb=arcpy.env.workspace, out_fc_name=None):
    """Joins a CSV file to a feature class and saves the result as a new feature class in a geodatabase.

    This function creates a schema.ini file for the specified CSV file, converts the CSV into a 
    geodatabase table, performs a join with the input feature class, and copies the joined 
    features into a new feature class. The function can also handle appending to an existing 
    schema.ini file or creating a new one if it does not exist.

    Args:
        in_fc (str): The path to the input feature class that the csv will join to.
        join_table (str): The path to the CSV file to be joined.
        in_field (str): The field name in the input feature class used for the join.
        join_field (str): The field name in the CSV file used for the join.
        join_field_len (int): The length of the join field in the CSV file (used for schema.ini).
        out_gdb (str, optional): The path to the output geodatabase where the new feature class will be saved. 
                                 Defaults to the current ArcPy workspace.
        out_fc_name (str, optional): The name for the output feature class. If None, defaults to '{table_name_noext}_fc'.
    
    Raises:
        Exception: If any of the ArcPy operations fail.

    Returns:
        Returns a new feature class with the joined data.
    """
        
    # create schema ini for csv
    table_path = os.path.dirname(join_table)
    table_name = os.path.basename(join_table)
    
    # Path to the schema.ini file
    schema_path = os.path.join(table_path, "schema.ini")

    # Create or overwrite schema.ini
    if os.path.exists(schema_path):
        action = "Appending to"
    else:
        action = "Creating"
        
    with open(schema_path, 'a') as schema_file:
        # Write the file-specific section in schema.ini
        schema_file.write(f"[{table_name}]\n")
        schema_file.write("Format=CSVDelimited\n")

        # define data type for col1
        schema_file.write(f"Col1={join_field} Text Width {join_field_len}\n")

        schema_file.write("\n")
        
    print(f'{action} schema.ini')
            
    # define table name
    table_name_noext = os.path.splitext(table_name)[0]
    
    # Convert CSV to GDB table using arcpy.conversion.TableToTable
    arcpy.conversion.TableToTable(
        in_rows=join_table,
        out_path=out_gdb,
        out_name=table_name_noext
    )
    print('converted csv to gdb table')
    # Make feature layer from input feature class
    layer_name = "temp_join"
    arcpy.management.MakeFeatureLayer(in_fc, layer_name)
    
    # Perform the join
    arcpy.management.AddJoin(
                in_layer_or_view=layer_name,
                in_field=in_field,
                join_table=table_name_noext,
                join_field=join_field
             )

    if out_fc_name is None:
        out_fc_name=f'{table_name_noext}_fc'
        
    # Copy the joined feature class to a new feature class
    arcpy.management.CopyFeatures(
        in_features=layer_name,
        out_feature_class=os.path.join(out_gdb, out_fc_name)
    )
    
    # Remove join
    arcpy.management.Delete(layer_name)

    print(f"Joined {os.path.basename(join_table)} to {os.path.basename(in_fc)} and saved as {out_fc_name}.")

In [26]:
# EXAMPLE
join_csv_to_fc(
    in_fc=r'Z:\UHC_Data\Census2010_MetroPHL\Geodatabases\Census2010_MetroPHL.gdb\UTM18\CT10_PHL_pd',
    join_table=r'Z:\UHC_Data\Disadvantage_Indices\Workshop_Data\data\segregation_CT10_phl.csv',
    in_field='GEOID10',
    join_field='GEOID10',
    join_field_len=11,
    out_fc_name=None)

Appending to schema.ini
converted csv to gdb table
Joined segregation_CT10_phl.csv to CT10_PHL_pd and saved as segregation_CT10_phl_fc.
